In [1]:
from theano import tensor as T
from theano import function

import numpy as np

### known_grads

`known_grads` is a dictionary that specifies the gradient of the cost w.r.t. to some variables, instead of the ones automatically derived from the cost. The cost can even be omitted if a seperating set is known. 

In [2]:
x = T.vector('x')
y = x ** 3 + x ** 2
z = 6 * y + x

# normal gradient
g1 = T.grad(z.mean(), x)

# dz/dx = dz/dy * dy/dx + d(x)/dx = (2 * x) * (3 * (x ** 2) + 2 * x)
g2 = T.grad(z.mean(), x, known_grads={y : 2 * x})

f1 = function([x], [y, z, g1])
f2 = function([x], [y, z, g2])

x_val = np.arange(2, dtype='float32')
print f1(x_val)
print f2(x_val)

[array([ 0.,  2.], dtype=float32), array([  0.,  13.], dtype=float32), array([  0.5,  15.5], dtype=float32)]
[array([ 0.,  2.], dtype=float32), array([  0.,  13.], dtype=float32), array([  0.5,  10.5], dtype=float32)]


### compute_test_value
In normal execution, the exception is only raised when it tries to execute the call to function. Thus, it's hard to see which line cause the error. `compute_test_value` allows you to plug in a test value in order to detect the first invalid expression.

In [ ]:
from theano import config

config.compute_test_value = 'raise'

a = T.vector()
a.tag.test_value = np.ones((3,)).astype(a.dtype)
b = T.log(a)
c = T.nnet.sigmoid(b)
d = T.sqrt(c)
e = T.concatenate((d, c), axis=0)
f = b * c * d
g = e + f # the first bad line
h = g / c
fn = function([a], h)
fn(np.ones((3,)).astype(a.dtype))